## *1. Preprocessing Notebook*

### Libraries


In [ ]:
from PIL import Image, ImageSequence
import cv2
import numpy as np
import os
import string
import torch
import glob


#### Directories

In [ ]:
dir_Acquisition = "\path\to\acquisition\directory"
dir_tiffLayers= "\path\to\tifflayers\directory"
dir_focused= "path\to\focused\layers\directory"
dir_cropped = "\path\to\cropped\slices\directory"

## Functions

#### Count files

In [ ]:
#Function to count files in a directory
def count_files(folder_path):
    count = 0
    for _, _, files in os.walk(folder_path):
        count += len(files)
    return count

#### Load images from folder

In [ ]:
#Function to load images from a directory
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

#### Read mulitlayered .tiff files and export focused layer

In [ ]:
#Function to Read multi-layered .tiff files and save the layer with highest sharpness score
def Tiff_ToStacks(dir_Acquisition,dir_tiffLayers,dir_focused):

    list_sharpness = []
    copy_list= []
    j = 0

    #Loop images in directorz acquisition
    for image in os.listdir(dir_Acquisition):
      os.chdir(dir_Acquisition)
      tiffopen = Image.open(image)
      j = j+1
      os.chdir(dir_tiffLayers)
      #Loop layers of each image
      for i, page in enumerate(ImageSequence.Iterator(tiffopen)):
        page.mode = 'I'
        page.point(lambda i:i*(1./256)).convert('L').save(str(j)+str(i)+'.jpeg')
        #print(str(j)+str(i))

        #find the focused page
          #Navigate to diretory with .tiffLayers
          #Load layers from Image_i to a collection
        collection  = load_images_from_folder(dir_tiffLayers)
        list_sharpness = []
        copy_list= []
        #Loop the collection
      for w in collection:
        gray = cv2.cvtColor(w, cv2.COLOR_BGR2GRAY)
        array = np.asarray(gray)
        gy, gx = np.gradient(array)
        gnorm = np.sqrt(gx**2 + gy**2)
        sharpness = np.average(gnorm)
        list_sharpness.append(sharpness)

        #copy the list of sharpness evaluation to sort and find the sharpest layer
      copy_list = list_sharpness.copy()
      copy_list.sort()
      #print(copy_list)
      #print(copy_list[list_len-1])

      #sharpest image is the layer with highest sharpness score which is the last element of the sorted array
      sharp= copy_list[len(copy_list)-1]
      s= list_sharpness.index(sharp)
      #print(copy_list)
      #print(list_sharpness)
      #print(s)
      #after getting the index of sharpest layer, save it from collection to focused images directory
      img_to_save = collection[s]
      img_path_name = dir_focused+'/'+str(i)+str(j)+'.jpeg'
      cv2.imwrite(img_path_name,img_to_save)
      print(img_path_name)
      #Delete all files in .tiff layers directory to prepare for loading the next image
      for file in os.listdir(dir_tiffLayers):
        if file.endswith('.jpeg'):
          os.remove(file)

#### Sliding window

In [ ]:
#Function to crop image into slices with Neubauer chamber grid size
# using a sliding window looping over the image
def Sliding_Window(image, stepSize, windowSize, e_num):
  #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  #Neubauer Grid size in pixels as calculated from image resolution
  h = 1440
  w = 1440
  #start with a small buffer
  #Step size is input from function call
  for j in range(100, image.shape[0]-h, stepSize):
    for i in range(100, image.shape[1]-w, stepSize):
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      roi = gray[j:j+ windowSize[1], i:i+ windowSize[0]]
      cv2.imwrite(str(i)+str(j)+str(e_num)+".png", roi)

## Preprocessing

#### Find focused stack

In [ ]:
#Function Call
Tiff_ToStacks(dir_Acquisition,dir_tiffLayers,dir_focused)

C:\CellCount\YOLO\OfflineYOLO\Focused/101.jpeg
C:\CellCount\YOLO\OfflineYOLO\Focused/102.jpeg


#### Sliding window crop

In [ ]:
#Load focused images into a collection
collection_tocrop  = load_images_from_folder(dir_focused)

#Crop Slices with sliding window
#Navigate to folder where cropped images will be saved
os.chdir(dir_cropped)
for no, img in enumerate(collection_tocrop):
  #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  Sliding_Window(img, 400, [1440,1440],no)
